In [1]:
# The magic commands below allow reflecting the changes in an imported module without restarting the kernel.
%load_ext autoreload
%autoreload 2
import sys
print(f'Python version: {sys.version.splitlines()[0]}')
print(f'Environment: {sys.exec_prefix}')

Python version: 3.7.4 (default, Aug 13 2019, 20:35:49) 
Environment: /home/keceli/.conda/envs/jhub_demo


In [33]:
# %load https://raw.githubusercontent.com/keceli/ezHPC/main/ez_balsam.py

def add_pgsql_path(path='/soft/datascience/balsam/pgsql/bin/'):
    """
    Add PostgreSQL directory to the path
    """
    import os
    if path not in os.environ['PATH']:
        os.environ['PATH'] += os.pathsep + path
    return

def check_pgsql(pgsql_exe='pg_ctl'):
    """
    Check PostgreSQL executable and version
    Balsam requires PostgreSQL version 9.6.4 or newer to be installed.
    TODO: Validate version
    """
    import os
    import shutil
    mypg_ctl = shutil.which(pgsql_exe)
    if mypg_ctl:
        print('PostgreSQL found: ', mypg_ctl)
        # pg_version = !$mypg_ctl --version
        pg_version = os.popen(f'{mypg_ctl} --version')
        print('PostgreSQL version: ', pg_version.read().split()[-1])
        return True
    else:
        print('PostgreSQL not found. Add PostgreSQL directory to the PATH')
        return False

def load_balsam():
    """
    Check Balsam module and print location and version
    """
    try:
        import balsam
        import os
        print('Balsam found: ', balsam.__file__)
        print('Balsam version: ', balsam.__version__)
        os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
        add_pgsql_path()
        check_pgsql()
    except Exception as e:
        print('🛑 Exception caught')
        print(e, '\n')
        print('Make sure Balsam is installed and you are using the right kernel/environment')
    return
    
def get_databases(verbose=True):
    """
    Return balsam databases. If verbose, print.
    """
    from balsam import django_config
    from balsam.django_config.db_index import refresh_db_index
    from ipywidgets import interact
    import os
    databasepaths = []
    try:
        databasepaths.extend(refresh_db_index())
        if verbose:
            print(f'There are {len(databasepaths)} Balsam databases available:')
            for i,db in enumerate(databasepaths):
                print(f'{i}: {db}')
    except Excpetion as e:
        print('🛑 Exception caught during balsam.django_config.db_index.refresh_db_index:')
        print(e, '\n')
    return databasepaths 
    
def activate_database(db=''):
    """
    Activates Balsam database by setting the BALSAM_DB_PATH environment variable.
    """
    import os
    os.environ["BALSAM_DB_PATH"] = db
    print(f'Selected database: {os.environ["BALSAM_DB_PATH"]}')
    return
    
def i_activate_database():
    """
    Activates Balsam database by setting the index of BALSAM_DB_PATH environment variable
    from the dropdown list.
    """
    from ipywidgets import interact
    databasepaths = get_databases()
    interact(activate_database,db=[(i,db) for i,db in enumerate(databasepaths)])
    return

def get_apps(verbose=True):
    """
    Return apps in the balsam database. If verbose, print.
    """
    from balsam.core.models import ApplicationDefinition as App
    from balsam.scripts import postgres_control
    import os
    try:
        apps = App.objects.all()
        if verbose:
            print(f'Found {len(apps)} apps in {os.environ["BALSAM_DB_PATH"]}:')
            for i,app in enumerate(apps):
                print(f'{i}: {app.name}')
        return apps
    except Exception as e:
        if 'could not connect to server' in str(e):
            print('🛑 Server exception caught:')
            print(e,'\n')
            print(f'Trying to restart the Balsam server {os.environ["BALSAM_DB_PATH"]} ...')
            try:
                postgres_control.start_main(os.environ["BALSAM_DB_PATH"])
            except Exception as e:
                print('Exception caught during restart:')
                print(e,'\n') 
        elif 'exit status 127' in str(e):
            print('🛑 Exception 127 caught:')
            print(e, '\n')   
            print('Checking postgresql')
            if not check_pgsql():
                print('Trying to add postgresql to the path')
                add_pgsql_path()
                if check_pgsql():
                    print('postgresql added, try again')
                else:
                    print('Unsuccessful, you need to add postgresql to the path manually')
        else:
            print('🛑 Unknown exception caught:')
            print('You may need to restart Balsam server on terminal')
            print(e,'\n')
        return None
    
def i_show_apps():
    """
    Show apps saved in the Balsam database
    """
    import os
    from ipywidgets import widgets, Layout
    from IPython.display import display, clear_output
    children = [widgets.Textarea(value=str(app), layout=Layout(flex= '1 1 auto', width='400px',height='200px')) 
                        for app in apps]
    tab = widgets.Accordion(children=children,layout=Layout(flex= '1 1 auto', width='500px',height='auto'))
    for i,app in enumerate(apps):
        tab.set_title(i, app.name)
    print(f'Apps in the Balsam database {os.environ["BALSAM_DB_PATH"]}:')
    display(tab)
    return
    
def save_app(name, executable, description='', envscript='', preprocess='', postprocess=''):
    """
    Adds a new app with the given properties to the balsam database.
    Parameters
    ----------
    name: str, name of the app
    executable: str, path to the executable
    description: str, info about the app
    preprocess: str, path to the preprocessing script
    postprocess: str, path to the postprocessing script
    """
    from balsam.core.models import ApplicationDefinition as App
    import shutil
    import os
    newapp = App()
    if App.objects.filter(name=name).exists():
        print(f"An application named {name} already exists")
        return
    else:
        newapp.name        = name
        newapp.executable  = executable
        newapp.description = description
        newapp.envscript   = envscript
        newapp.preprocess  = preprocess
        newapp.postprocess = postprocess
        appexe = shutil.which(executable)
        if appexe:        
            print(f'{appexe} is found')
            newapp.save()
            print(f'{newapp.name} added to the balsam database {os.environ["BALSAM_DB_PATH"]}.')
        else:
            print('{executable} is not found')
    return

def i_save_app():
    """
    Adds a new app to the balsam database with the given properties to the balsam database.
    """
    from ipywidgets import interact_manual
    import os
    print(f'Balsam database: {os.environ["BALSAM_DB_PATH"]}')
    im = interact_manual(save_app, name='', executable='')
    app_button = im.widget.children[6]
    app_button.description = 'save app'
    return

def delete_app(name):
    """
    Delete Balsam app with the given name
    Note: All apps with the same name will be deleted
    """
    from balsam.core.models import ApplicationDefinition as App
    if App.objects.filter(name=name).exists():
        app = App.objects.filter(name=name)
        app.delete()
        print(f'{name} app deleted.')
    else:
        print(f'{name} app not found.' )
    return

def i_delete_app():
    """
    Delete selected Balsam app
    """    
    from balsam.core.models import ApplicationDefinition as App
    from ipywidgets import widgets, interact_manual
    from IPython.display import display, clear_output
    import os
    print(f'Balsam database: {os.environ["BALSAM_DB_PATH"]}')
    allapps = [app.name for app in App.objects.all()]
    idelete = widgets.Button(
                    value=False,
                    description='delete app',
                    disabled=False,
                    button_style='danger',
                    tooltip='Delete app',
                    icon='')
    iapps = widgets.Dropdown(options=allapps, description='app')
    output = widgets.Output()
    display(iapps,idelete,output)
    def delete_clicked(b):
        with output:
            clear_output()
            delete_app(iapps.value)
    idelete.on_click(delete_clicked)
    return   

def save_job(name, workflow, application, description='', 
            args='', num_nodes=1, ranks_per_node=1,
            cpu_affinity='depth', data={}, environ_vars=''):
    """
    Adds and returns a new job with the given properties
    """
    from balsam.launcher.dag import BalsamJob
    from balsam.core.models import ApplicationDefinition as App
    import os
    job                = BalsamJob()
    job.name           = name
    job.workflow       = workflow
    job.application    = application
    job.description    = description
    job.args           = args
    job.num_nodes      = num_nodes
    job.ranks_per_node = ranks_per_node
    job.cpu_affinity   = cpu_affinity
    job.environ_vars   = environ_vars
    job.data           = {}
    job.save()
    print(f'{job.name} {job.job_id} added to the balsam database {os.environ["BALSAM_DB_PATH"]}.')
    return job

def i_save_job():
    """
    Adds and returns a new job with the given properties
    """
    from ipywidgets import interact, interact_manual
    from IPython.display import display, clear_output
    from balsam.core.models import ApplicationDefinition as App
    from ipywidgets import widgets
    import os
    print(f'Balsam database: {os.environ["BALSAM_DB_PATH"]}')
    apps = App.objects.all()
    appnames = [app.name for app in apps]
    isave = widgets.ToggleButton(
                value=False,
                description='save job',
                disabled=False,
                button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                tooltip='save job to the balsam database',
                icon='') 
    im = interact_manual(save_job, name='', workflow='', application=appnames, description='', 
              args='', num_nodes=range(1,4394), ranks_per_node=range(1,256),
              cpu_affinity=['depth','none'],data={},environ_vars='')
    app_button = im.widget.children[10]
    app_button.description = 'save job'
    return

def show_job_info(job_id='',show_output=False):
    """
    Prints verbose job info for a given job id.
    Parameters
    ----------
    job_id: str, Partial or full Balsam job id.
    """
    from balsam.launcher.dag import BalsamJob as Job
    import pathlib
    jobs = Job.objects.all().filter(job_id__contains=job_id)
    if len(jobs) == 1:
        thejob = jobs[0]
        print(jobs[0])
        if show_output:
            output = f'{thejob.working_directory}/{thejob.name}.out'
            if pathlib.Path(output).is_file():
                with open(output) as f:
                    out = f.read()
                print(f'Output file {output} content:')
                print(out)
            else:
                print(f'{output} not found.')
                print(f'Job state: {thejob.state}')
                if thejob.state =='CREATED':
                    print('The job has not run yet.')
    elif len(jobs) == 0:
        print('No matching jobs')
    else:
        print(f'{len(jobs)} jobs matched, enter full id.')
        print('Matched jobs:')
        for job in jobs:
            print(f'{job.name}: {job.job_id} ')
    return

def i_show_job_info():
    """Show the job verbose information for a given job id"""
    from ipywidgets import interact
    from IPython.display import display, clear_output
    interact(show_job_info)
    return

def list_jobs(state='ALL',workflow='ALL',app='ALL',name=''):
    """
    List Balsam jobs with the given properties
    """
    from balsam.launcher.dag import BalsamJob as Job
    from balsam.core.models import ApplicationDefinition as App
    jobs = Job.objects.all()
    print(f'Total number of jobs: {len(jobs)}')
    if state != 'ALL':
        jobs = jobs.filter(state=state)
    if workflow != 'ALL':
        jobs = jobs.filter(workflow=workflow)
    if app != 'ALL':
        jobs = jobs.filter(application=app)
    if name:
        jobs = jobs.filter(name__icontains=name)
    print(f'Selected number of jobs: {len(jobs)}')
    if len(jobs) > 0: 
        t = '{:<20}'.format('Name')
        t += ' {:>8}'.format('Nodes')
        t += ' {:>12}'.format('Ranks')
        t += ' {:^8}'.format('ID')
        if state =='JOB_FINISHED':
            t += '{:>12}'.format('Runtime')
        elif state =='ALL':
            t += '{:>15}'.format('State')
        print(t)
        for job in jobs:
            s = '{:<20.15}'.format(job.name)
            s += ' {:>8}'.format(job.num_nodes)
            s += ' {:>12}'.format(job.num_ranks)
            s += '  {:>8}'.format(str(job.job_id).split('-')[0])            

            if state =='JOB_FINISHED':
                s += '{:>12.3f}'.format(job.runtime_seconds)
            elif state =='ALL':
                s += '{:>15}'.format(job.state)
            print(s)
    return
            
def i_list_jobs():
    """
    List Balsam jobs with the given properties
    """
    from balsam.launcher.dag import BalsamJob as Job
    from balsam.core.models import ApplicationDefinition as App
    from ipywidgets import widgets, interact
    from IPython.display import display, clear_output

    allstates = ['ALL',
                 'CREATED',
                 'AWAITING_PARENTS',
                 'READY',
                 'STAGED_IN',
                 'PREPROCESSED',
                 'RUNNING',
                 'RUN_DONE',
                 'POSTPROCESSED',
                 'JOB_FINISHED',
                 'RUN_TIMEOUT',
                 'RUN_ERROR',
                 'RESTART_READY',
                 'FAILED',
                 'USER_KILLED']
    allworkflows = [wf['workflow'] for wf in Job.objects.order_by().values('workflow').distinct()]
    allworkflows.append('ALL')
    allapps = [app.name for app in App.objects.all()]
    allapps.append('ALL')
    ilist = widgets.Button(
                    value=False,
                    description='list jobs',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='List selected jobs',
                    icon='') 
    im = interact(list_jobs, state=allstates, workflow=allworkflows, 
                 app=allapps, name='')
    return

def delete_jobs(state='ALL',workflow='ALL',app='ALL',name='', confirm=False):
    """
    Delete Balsam jobs with the given properties
    """
    jobs = Job.objects.all()
    print(f'Total number of jobs: {len(jobs)}')
    if state != 'ALL':
        jobs = jobs.filter(state=state)
    if workflow != 'ALL':
        jobs = jobs.filter(workflow=workflow)
    if app != 'ALL':
        jobs = jobs.filter(application=app)
    if name:
        jobs = jobs.filter(name__icontains=name)
    print(f'Selected number of jobs: {len(jobs)}')
    if len(jobs) > 0: 
        t = '{:<20}'.format('Name')
        t += ' {:>8}'.format('Nodes')
        t += ' {:>12}'.format('Ranks')
        t += ' {:^8}'.format('ID')
        if state =='JOB_FINISHED':
            t += '{:>12}'.format('Runtime')
        elif state =='ALL':
            t += '{:>15}'.format('State')
        print(t)
        for job in jobs:
            s = '{:<20.15}'.format(job.name)
            s += ' {:>8}'.format(job.num_nodes)
            s += ' {:>12}'.format(job.num_ranks)
            s += '  {:>8}'.format(str(job.job_id).split('-')[0])            

            if state =='JOB_FINISHED':
                s += '{:>12.3f}'.format(job.runtime_seconds)
            elif state =='ALL':
                s += '{:>15}'.format(job.state)
            print(s)
        if confirm:
            try:
                for job in jobs:
                    print(f"Deleting {job.name} {str(job.job_id).split('-')[0]}")
                    job.delete()
                print(f'Deleted {len(jobs)} jobs')
            except Exception as e:
                print('Exception caught while deleting the selected jobs:')
                print(e)
    return
    
def i_delete_jobs():
    """
    Delete Balsam jobs with the given properties
    """
    from balsam.launcher.dag import BalsamJob as Job
    from balsam.core.models import ApplicationDefinition as App
    from ipywidgets import widgets, fixed
    from IPython.display import display, clear_output

    allstates = ['ALL',
                 'CREATED',
                 'AWAITING_PARENTS',
                 'READY',
                 'STAGED_IN',
                 'PREPROCESSED',
                 'RUNNING',
                 'RUN_DONE',
                 'POSTPROCESSED',
                 'JOB_FINISHED',
                 'RUN_TIMEOUT',
                 'RUN_ERROR',
                 'RESTART_READY',
                 'FAILED',
                 'USER_KILLED']
    allworkflows = [wf['workflow'] for wf in Job.objects.order_by().values('workflow').distinct()]
    allworkflows.append('ALL')
    allapps = [app.name for app in App.objects.all()]
    allapps.append('ALL')
    iconfirm = widgets.Checkbox(value=False, description='confirm delete')
    ilist = widgets.Button(
                    value=False,
                    description='list jobs',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='List selected jobs',
                    icon='') 
    idelete = widgets.Button(
                    value=False,
                    description='DELETE',
                    disabled=False,
                    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='Deletes selected jobs',
                    icon='') # check
    istate = widgets.Dropdown(options=allstates,value='ALL',description='state')
    iworkflow = widgets.Dropdown(options=allworkflows,value='ALL',description='workflow')
    iapp = widgets.Dropdown(options=allapps,value='ALL',description='app')
    iname = widgets.Text(value='', description='name')
    output = widgets.Output()
    display(istate, iworkflow, iapp, iname, iconfirm, ilist, idelete, output)
    def delete_clicked(b):
        with output:
            clear_output()
            delete_jobs(state=istate.value,workflow=iworkflow.value,app=iapp.value,
                     name=iname.value, confirm=iconfirm.value)
    def list_clicked(b):
        with output:
            clear_output()
            list_jobs(state=istate.value,workflow=iworkflow.value,app=iapp.value,
                     name=iname.value)
    idelete.on_click(delete_clicked)
    ilist.on_click(list_clicked)
    return

def submit_jobs(project='',queue='debug-cache-quad',nodes=1,
           wall_minutes=30,job_mode='mpi',wf_filter='',
           save=False,submit=False):
    """
    Submits a job to the queue with the given parameters.
    Parameters
    ----------
    project: str, name of the project to be charged
    queue: str, queue name, can be: 'default', 'debug-cache-quad', 'debug-flat-quad', 'backfill'
    nodes: int, Number of nodes, can be an integer from 1 to 4096 depending on the queue.
    wall_minutes: int, max wall time in minutes, depends on the queue and the number of nodes, max 1440 minutes
    job_mode: str, Balsam job mode, can be 'mpi', 'serial'
    wf_filter: str, Selects Balsam jobs that matches the given workflow filter.
    """
    from balsam import setup
    setup()
    from balsam.service import service
    from balsam.core import models
    validjob = True
    QueuedLaunch = models.QueuedLaunch
    mylaunch = QueuedLaunch()
    mylaunch.project = project
    mylaunch.queue = queue
    mylaunch.nodes = nodes
    mylaunch.wall_minutes = wall_minutes
    mylaunch.job_mode = job_mode
    mylaunch.wf_filter = wf_filter
    mylaunch.prescheduled_only=False
    if queue.startswith('debug'):
        if wall_minutes > 60:
            validjob = False
            print(f'Max wall time for {queue} queue is 60 minutes')
        if nodes > 8:
            validjob = False
            print(f'Max number of nodes for {queue} queue is 8')
    else:
        if nodes < 128:
            validjob = False
            print(f'Min number of nodes for {queue} queue is 128')
    if save and validjob:
        mylaunch.save()
        print(f'Ready to submit')
        if submit:
            service.submit_qlaunch(mylaunch, verbose=True)
    
def i_submit_jobs():
    from ipywidgets import interact, widgets
    inodes = widgets.BoundedIntText(value=1, min=1, max=4394, step=1, description='nodes', disabled=False)
    iwall_minutes = widgets.BoundedIntText(value=10, min=10, max=1440, step=30, description='wall minutes', disabled=False)
    isave = widgets.Checkbox(value=False,description='save', indent=True)
    isubmit = widgets.ToggleButton(
                    value=False,
                    description='submit',
                    disabled=False,
                    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='submit job',
                    icon='') # ('check')
    im = interact(submit_jobs, project='',queue=['debug-flat-quad','debug-cache-quad','default', 'backfill'],
                         nodes=inodes, wall_minutes=iwall_minutes, job_mode=['mpi','serial'],
                         wf_filter='', save=isave, submit=isubmit)


Overwriting ezHPC/ez_balsam.py


In [4]:
load_balsam()

Balsam found:  /home/keceli/.conda/envs/jhub_demo/lib/python3.7/site-packages/balsam/__init__.py
Balsam version:  0.3.8
PostgreSQL found:  /soft/datascience/balsam/pgsql/bin/pg_ctl
PostgreSQL version:  9.6.12


In [8]:
#help('modules')
#help('modules mpi4py')


Here is a list of modules whose name or summary contains 'mpi4py'.
If there are any, enter a module name to get more help.

mpi4py - This is the **MPI for Python** package.
mpi4py.MPI 
mpi4py.__main__ - Entry-point for ``python -m mpi4py ...``.
mpi4py.bench - Run MPI benchmarks and tests.
mpi4py.dl 
mpi4py.futures - Execute computations asynchronously using MPI processes.
mpi4py.futures.__main__ - Run Python code using ``mpi4py.futures``.
mpi4py.futures._base 
mpi4py.futures._lib - Management of MPI worker processes.
mpi4py.futures.aplus - Support for Future chaining.
mpi4py.futures.pool - Implements MPIPoolExecutor.
mpi4py.futures.server - Entry point for MPI workers.
mpi4py.run - Run Python code using ``mpi4py``.



0. Check the names of the existing environments
1. Select a name for the new environment & kernel.

In [4]:
!conda env list

# conda environments:
#
ffn                      /home/keceli/.conda/envs/ffn
fireworks_theta          /home/keceli/.conda/envs/fireworks_theta
jhub_balsam              /home/keceli/.conda/envs/jhub_balsam
jhub_chem                /home/keceli/.conda/envs/jhub_chem
jhub_chembox             /home/keceli/.conda/envs/jhub_chembox
jhub_data                /home/keceli/.conda/envs/jhub_data
jhub_demo4               /home/keceli/.conda/envs/jhub_demo4
jhub_demo5               /home/keceli/.conda/envs/jhub_demo5
jhub_ffn                 /home/keceli/.conda/envs/jhub_ffn
jhub_ffn3                /home/keceli/.conda/envs/jhub_ffn3
jhub_fortran             /home/keceli/.conda/envs/jhub_fortran
jhub_py3.8               /home/keceli/.conda/envs/jhub_py3.8
jhub_python2             /home/keceli/.conda/envs/jhub_python2
jhub_python2_test        /home/keceli/.conda/envs/jhub_python2_test
jhub_tensorflow          /home/keceli/.conda/envs/jhub_tensorflow
jhub_test                /home/keceli/.conda/envs

In [5]:
ENVNAME="jhub_demo"

3. Create a new environment with a clone of the base environment

In [6]:
%%time
!conda create -y --name $ENVNAME --clone base

Source:      /opt/anaconda3x
Destination: /home/keceli/.conda/envs/jhub_demo
The following packages cannot be cloned out of the root environment:
 - defaults/linux-64::conda-4.8.1-py37_0
 - defaults/linux-64::conda-build-3.18.9-py37_3
 - defaults/linux-64::conda-env-2.6.0-1
Packages: 464
Files: 782
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate jhub_demo
#
# To deactivate an active environment, use
#
#     $ conda deactivate

CPU times: user 2.09 s, sys: 460 ms, total: 2.55 s
Wall time: 2min 44s


3.5 If you want change the env name, you may need to remove the environment

In [7]:
%%time
#!conda env remove --name jhub_demo5


Remove all packages in environment /home/keceli/.conda/envs/jhub_demo5:

CPU times: user 566 ms, sys: 93 ms, total: 659 ms
Wall time: 50 s


4. Install a new package

In [8]:
%%time
!source activate $ENVNAME; pip install -v --no-cache-dir balsam-flow

Created temporary directory: /tmp/pip-ephem-wheel-cache-7yzahzvh
Created temporary directory: /tmp/pip-req-tracker-e8ddg8cf
Created requirements tracker '/tmp/pip-req-tracker-e8ddg8cf'
Created temporary directory: /tmp/pip-install-t07zo0dp
1 location(s) to search for versions of balsam-flow:
* https://pypi.org/simple/balsam-flow/
Getting page https://pypi.org/simple/balsam-flow/
Found index url https://pypi.org/simple
Getting credentials from keyring for https://pypi.org/simple
Getting credentials from keyring for pypi.org
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /simple/balsam-flow/ HTTP/1.1" 200 936
Analyzing links from page https://pypi.org/simple/balsam-flow/
  Found link https://files.pythonhosted.org/packages/61/3c/e3f6c11e193cda14472420e31bf7c66a882bf96a09fe313933a4c8e226fb/balsam-flow-0.3.5.tar.gz#sha256=9afa525417c5809fbd9b39b7f7e6fb43898bff816f89b5e1cea8e6f75b3649b5 (from https://pypi.org/simple/balsam-flow/) (requires-python:>=3.6), version: 

     |██████████▍                 

     |████████████████████▊     

     |████████████████████████████████| 10.8MB 18.8MB/s 


  Added sphinx-rtd-theme from https://files.pythonhosted.org/packages/c3/86/1addf25a238bbd8466bb099f23d9a9f13494b22b37b44f6c41a778b8730f/sphinx_rtd_theme-0.5.0-py2.py3-none-any.whl#sha256=373413d0f82425aaa28fb288009bf0d0964711d347763af2f1b65cafcb028c82 (from balsam-flow) to build tracker '/tmp/pip-req-tracker-e8ddg8cf'
  Removed sphinx-rtd-theme from https://files.pythonhosted.org/packages/c3/86/1addf25a238bbd8466bb099f23d9a9f13494b22b37b44f6c41a778b8730f/sphinx_rtd_theme-0.5.0-py2.py3-none-any.whl#sha256=373413d0f82425aaa28fb288009bf0d0964711d347763af2f1b65cafcb028c82 (from balsam-flow) from build tracker '/tmp/pip-req-tracker-e8ddg8cf'
1 location(s) to search for versions of django:
* https://pypi.org/simple/django/
Getting page https://pypi.org/simple/django/
Found index url https://pypi.org/simple
Getting credentials from keyring for https://pypi.org/simple
Getting credentials from keyring for pypi.org
https://pypi.org:443 "GET /simple/django/ HTTP/1.1" 200 36658
Analyzing links fr

...

  Added psycopg2-binary from https://files.pythonhosted.org/packages/6d/45/c519a5cfac05e14b1ccb242138915855293199840598e087b935ba1d86bc/psycopg2_binary-2.8.6-cp37-cp37m-manylinux1_x86_64.whl#sha256=6a32f3a4cb2f6e1a0b15215f448e8ce2da192fd4ff35084d80d5e39da683e79b (from balsam-flow) to build tracker '/tmp/pip-req-tracker-e8ddg8cf'
  Removed psycopg2-binary from https://files.pythonhosted.org/packages/6d/45/c519a5cfac05e14b1ccb242138915855293199840598e087b935ba1d86bc/psycopg2_binary-2.8.6-cp37-cp37m-manylinux1_x86_64.whl#sha256=6a32f3a4cb2f6e1a0b15215f448e8ce2da192fd4ff35084d80d5e39da683e79b (from balsam-flow) from build tracker '/tmp/pip-req-tracker-e8ddg8cf'
1 location(s) to search for versions of asgiref:
* https://pypi.org/simple/asgiref/
Getting page https://pypi.org/simple/asgiref/
Found index url https://pypi.org/simple
Getting credentials from keyring for https://pypi.org/simple
Getting credentials from keyring for pypi.org
https://pypi.org:443 "GET /simple/asgiref/ HTTP/1.1" 200 

1 location(s) to search for versions of sqlparse:
* https://pypi.org/simple/sqlparse/
Getting page https://pypi.org/simple/sqlparse/
Found index url https://pypi.org/simple
Getting credentials from keyring for https://pypi.org/simple
Getting credentials from keyring for pypi.org
https://pypi.org:443 "GET /simple/sqlparse/ HTTP/1.1" 200 3666
Analyzing links from page https://pypi.org/simple/sqlparse/
  Found link https://files.pythonhosted.org/packages/a9/82/78f60ca2d543578193a25679e2f7af9323fa6bce3689090c3f803dab8b51/sqlparse-0.1.0.tar.gz#sha256=760c0454fbc3c9ecffbddc5d08627cfff0aac01319743cfd894948024ef9c598 (from https://pypi.org/simple/sqlparse/), version: 0.1.0
  Found link https://files.pythonhosted.org/packages/39/9c/47b05772980c3341460c38e0c82620800ee8e41eca5d704c4a2662f5c19c/sqlparse-0.1.1.tar.gz#sha256=6049385a7032628c9a9bc2ac578ce1826e4179b0a8a9f33b535f022fabd8d3ea (from https://pypi.org/simple/sqlparse/), version: 0.1.1
  Found link https://files.pythonhosted.org/packages/8c




  changing mode of /home/keceli/.conda/envs/jhub_demo/bin/sqlformat to 755

  changing mode of /home/keceli/.conda/envs/jhub_demo/bin/django-admin to 755


  changing mode of /home/keceli/.conda/envs/jhub_demo/bin/balsam to 755
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-e8ddg8cf'
1 location(s) to search for versions of pip:
* https://pypi.org/simple/pip/
Getting page https://pypi.org/simple/pip/
Found index url https://pypi.org/simple
Getting credentials from keyring for https://pypi.org/simple
Getting credentials from keyring for pypi.org
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /simple/pip/ HTTP/1.1" 200 14495
Analyzing links from page https://pypi.org/simple/pip/
  Found link https://files.pythonhosted.org/packages/3d/9d/1e313763bdfb6a48977b65829c6ce2a43eaae29ea2f907c8bbef024a7219/pip-0.2.tar.gz#sha256=88bb8d029e1bf4acd0e04d300104b7440086f94cc1ce1c5c3c31e3293aee1f81 (from https://pypi.org/simple/pip/), version: 0.2
  Found link htt

  Found link https://files.pythonhosted.org/packages/51/5f/802a04274843f634469ef299fcd273de4438386deb7b8681dd059f0ee3b7/pip-19.1.tar.gz#sha256=d9137cb543d8a4d73140a3282f6d777b2e786bb6abb8add3ac5b6539c82cd624 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.1
  Found link https://files.pythonhosted.org/packages/5c/e0/be401c003291b56efc55aeba6a80ab790d3d4cece2778288d65323009420/pip-19.1.1-py2.py3-none-any.whl#sha256=993134f0475471b91452ca029d4390dc8f298ac63a712814f101cd1b6db46676 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.1.1
  Found link https://files.pythonhosted.org/packages/93/ab/f86b61bef7ab14909bd7ec3cd2178feb0a1c86d451bc9bccd5a1aedcde5f/pip-19.1.1.tar.gz#sha256=44d3d7d3d30a1eb65c7e5ff1173cdf8f7467850605ac7cc3707b6064bddd0958 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.1.1
  Found link https://files.python

CPU times: user 296 ms, sys: 105 ms, total: 400 ms
Wall time: 22.6 s


5. Insall the kernel for Jupyter

In [9]:
%%time
!source activate $ENVNAME;python -m ipykernel install --user --name $ENVNAME

/home/keceli/.conda/envs/jhub_demo/lib/python3.7/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  from ipykernel import kernelapp as app
Installed kernelspec jhub_demo in /gpfs/mira-home/keceli/.local/share/jupyter/kernels/jhub_demo
CPU times: user 77.8 ms, sys: 18.6 ms, total: 96.4 ms
Wall time: 7.77 s


* Refresh the browser
* Select the new `Kernel` from the top dropdownlist
* When you need to install another package, you only need to run step 4

In [ ]:
%%time
ENVNAME='jhub_demo' 
!source activate $ENVNAME; conda install -y -c conda-forge rise

In [5]:
i_activate_database()

There are 12 Balsam databases available:
0: /lus/theta-fs0/projects/datascience/keceli/valence_balsam/balsamdb
1: /lus/theta-fs0/projects/datascience/keceli/balsam/nwchem_demo/db
2: /lus/theta-fs0/projects/datascience/keceli/balsam/simint/simint_db
3: /lus/theta-fs0/projects/datascience/keceli/balsam/balsamdb_general
4: /lus/theta-fs0/projects/datascience/keceli/balsam/jupyter_test
5: /lus/theta-fs0/projects/connectomics_aesp/balsam_database
6: /lus/theta-fs0/projects/datascience/keceli/container/scaling_test/connectomics
7: /lus/theta-fs0/projects/datascience/keceli/nwx/nwx_db
8: /home/keceli/test_balsam
9: /gpfs/mira-home/keceli/test_balsam
10: /home/keceli/test_balsam_theta
11: /gpfs/mira-home/keceli/test_balsam_theta


interactive(children=(Dropdown(description='db', options=((0, '/lus/theta-fs0/projects/datascience/keceli/vale…

In [13]:
apps = get_apps();

Found 5 apps in /gpfs/mira-home/keceli/test_balsam_theta:
0: SayHello [SayHello | 1]
1: sleep [sleep | 2]
2: host [host | 3]
3: cat [cat | 4]
4: delete [delete | 17]


In [32]:
i_show_apps()

Apps in the Balsam database /gpfs/mira-home/keceli/test_balsam_theta:


Accordion(children=(Textarea(value='Application 1:\n-----------------------\nname:                           S…

In [8]:
i_save_app()

Balsam database: /gpfs/mira-home/keceli/test_balsam_theta


interactive(children=(Text(value='', description='name'), Text(value='', description='executable'), Text(value…

In [9]:
i_list_jobs()

interactive(children=(Dropdown(description='state', options=('ALL', 'CREATED', 'AWAITING_PARENTS', 'READY', 'S…

In [10]:
i_save_job()

Balsam database: /gpfs/mira-home/keceli/test_balsam_theta


interactive(children=(Text(value='', description='name'), Text(value='', description='workflow'), Dropdown(des…

In [11]:
i_show_job_info()

interactive(children=(Text(value='', description='job_id'), Checkbox(value=False, description='show_output'), …

In [12]:
i_submit_jobs()

interactive(children=(Text(value='', description='project'), Dropdown(description='queue', index=1, options=('…